In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [2]:
raw_df = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.load("/user/itv019463/TrendyTechBigData/Week6/data/order_items/part-00000")

In [3]:
raw_df.show(5)

+---+---+----+---+------+------+
|_c0|_c1| _c2|_c3|   _c4|   _c5|
+---+---+----+---+------+------+
|  1|  1| 957|  1|299.98|299.98|
|  2|  2|1073|  1|199.99|199.99|
|  3|  2| 502|  5| 250.0|  50.0|
|  4|  2| 403|  1|129.99|129.99|
|  5|  4| 897|  2| 49.98| 24.99|
+---+---+----+---+------+------+
only showing top 5 rows



In [4]:
raw_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)



In [5]:
refined_df = raw_df.toDF("order_item_id", "order_id", "product_id", "quantity", "subtotal", "product_price")

In [6]:
refined_df.show(5)

+-------------+--------+----------+--------+--------+-------------+
|order_item_id|order_id|product_id|quantity|subtotal|product_price|
+-------------+--------+----------+--------+--------+-------------+
|            1|       1|       957|       1|  299.98|       299.98|
|            2|       2|      1073|       1|  199.99|       199.99|
|            3|       2|       502|       5|   250.0|         50.0|
|            4|       2|       403|       1|  129.99|       129.99|
|            5|       4|       897|       2|   49.98|        24.99|
+-------------+--------+----------+--------+--------+-------------+
only showing top 5 rows



In [7]:
refined_df.printSchema()

root
 |-- order_item_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- subtotal: double (nullable = true)
 |-- product_price: double (nullable = true)



In [8]:
df1 = refined_df.drop("subtotal")

In [9]:
df1.show(5)

+-------------+--------+----------+--------+-------------+
|order_item_id|order_id|product_id|quantity|product_price|
+-------------+--------+----------+--------+-------------+
|            1|       1|       957|       1|       299.98|
|            2|       2|      1073|       1|       199.99|
|            3|       2|       502|       5|         50.0|
|            4|       2|       403|       1|       129.99|
|            5|       4|       897|       2|        24.99|
+-------------+--------+----------+--------+-------------+
only showing top 5 rows



In [10]:
df1.select("order_item_id", "order_id", "product_id").show(5)

+-------------+--------+----------+
|order_item_id|order_id|product_id|
+-------------+--------+----------+
|            1|       1|       957|
|            2|       2|      1073|
|            3|       2|       502|
|            4|       2|       403|
|            5|       4|       897|
+-------------+--------+----------+
only showing top 5 rows



In [11]:
from pyspark.sql.functions import *

In [12]:
df1.select("*", expr("product_price * quantity as subtotal")).show(5)

+-------------+--------+----------+--------+-------------+--------+
|order_item_id|order_id|product_id|quantity|product_price|subtotal|
+-------------+--------+----------+--------+-------------+--------+
|            1|       1|       957|       1|       299.98|  299.98|
|            2|       2|      1073|       1|       199.99|  199.99|
|            3|       2|       502|       5|         50.0|   250.0|
|            4|       2|       403|       1|       129.99|  129.99|
|            5|       4|       897|       2|        24.99|   49.98|
+-------------+--------+----------+--------+-------------+--------+
only showing top 5 rows



In [15]:
df1.selectExpr("*", "product_price * quantity as subtotal").show(5)

+-------------+--------+----------+--------+-------------+--------+
|order_item_id|order_id|product_id|quantity|product_price|subtotal|
+-------------+--------+----------+--------+-------------+--------+
|            1|       1|       957|       1|       299.98|  299.98|
|            2|       2|      1073|       1|       199.99|  199.99|
|            3|       2|       502|       5|         50.0|   250.0|
|            4|       2|       403|       1|       129.99|  129.99|
|            5|       4|       897|       2|        24.99|   49.98|
+-------------+--------+----------+--------+-------------+--------+
only showing top 5 rows



In [16]:
products_df = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.load("/public/trendytech/retail_db/products/part-00000")

In [18]:
products_df.show(5, truncate = False)

+---+---+---------------------------------------------+----+------+-------------------------------------------------------------------------------------+
|_c0|_c1|_c2                                          |_c3 |_c4   |_c5                                                                                  |
+---+---+---------------------------------------------+----+------+-------------------------------------------------------------------------------------+
|1  |2  |Quest Q64 10 FT. x 10 FT. Slant Leg Instant U|null|59.98 |http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy|
|2  |2  |Under Armour Men's Highlight MC Football Clea|null|129.99|http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat     |
|3  |2  |Under Armour Men's Renegade D Mid Football Cl|null|89.99 |http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat   |
|4  |2  |Under Armour Men's Renegade D Mid Football Cl|null|89.99 |http://im

In [19]:
products_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: string (nullable = true)



In [21]:
df1 = products_df.toDF("product_id", "product_category_id", "product_name", "product_description", "product_price", "product_image")

In [22]:
df1.show(5)

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|        59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|       129.99|http://images.acm...|
|         3|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         4|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               null|       199.99|http://images.acm...|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
only showing top 5 

In [23]:
df1.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_category_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_description: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_image: string (nullable = true)



In [26]:
df2 = df1.withColumn("product_price", expr("product_price * 1.2"))

In [28]:
df2.show(2)

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|       71.976|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|      155.988|http://images.acm...|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
only showing top 2 rows



In [34]:
new_df = df1.withColumn("product_price", expr("case when product_name like '%Nike%' then product_price * 1.2 when product_name like '%Armour%' then product_price * 1.1 else product_price end"))

In [35]:
new_df.show()

+----------+-------------------+--------------------+-------------------+------------------+--------------------+
|product_id|product_category_id|        product_name|product_description|     product_price|       product_image|
+----------+-------------------+--------------------+-------------------+------------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|             59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|142.98900000000003|http://images.acm...|
|         3|                  2|Under Armour Men'...|               null|            98.989|http://images.acm...|
|         4|                  2|Under Armour Men'...|               null|            98.989|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               null|            199.99|http://images.acm...|
|         6|                  2|Jordan Men's VI R...|               null|            134